In [1]:
import sentence_transformers  # dependency for sentence_feature_extractor.py
# check that we can use sentence_transformers
print(sentence_transformers.SentenceTransformer('all-MiniLM-L6-v2').encode(['hello world']).shape)
# import openai  # dependency for chatgpt.py
import evadb
cursor = evadb.connect(evadb_dir=("./eva_db")).cursor()

/home/oscar/.cache/pypoetry/virtualenvs/words-Cdd64V6x-py3.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/oscar/.cache/pypoetry/virtualenvs/words-Cdd64V6x-py3.10/lib/python3.10/site-packages/torch/cuda/__init__.py:138: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0


(1, 384)


In [2]:
# Register the function as embed text
cursor.query("""
DROP FUNCTION IF EXISTS EmbedText
""").df()
cursor.query("""
CREATE FUNCTION EmbedText IMPL './sentence_feature_extractor.py'
""").df()
cursor.query("""
SELECT EmbedText('hello world')
""").df()

11-19-2023 00:19:15 WARNING[drop_object_executor:drop_object_executor.py:_handle_drop_function:0086] Function EmbedText does not exist, therefore cannot be dropped.


,features
0,"[[-0.03447732, 0.031023178, 0.0067349477, 0.02..."


In [4]:
cursor.query("""
DROP TABLE IF EXISTS words
""").df()
cursor.query("""
CREATE TABLE words (
            word TEXT(0)
            )"""
).df()
cursor.query("""
SELECT * FROM words
""").df()


""


In [ ]:
# iterate thru ./top_english_words_lower_50000.txt

from pathlib import Path
full_text = Path('./top_english_words_lower_50000.txt').read_text()
words = full_text.split('\n')
print(words[:10])

from tqdm import tqdm   

for word in tqdm(words):
    bad_chars = '{}-;:"\''
    bad_chars_translation = str.maketrans(dict.fromkeys(bad_chars, ''))
    word = word.translate(bad_chars_translation)
    print(word)
    cursor.query(f"""
    INSERT INTO words (word) VALUES ('{word}')
    """).df()

In [2]:
# check that the words are in the table
cursor.query("""
SELECT * FROM words
""").df()

,_row_id,word
0,1,the
1,2,of
2,3,and
3,4,to
4,5,in
...,...,...
51581,51582,auth
51582,51583,quelled
51583,51584,splinting
51584,51585,empyema


In [10]:
# create a vector index using faiss
# this should take about 10m to run.
cursor.query("""
DROP INDEX IF EXISTS words_vector_idx
""").df()
cursor.query("""
CREATE INDEX words_vector_idx
ON words (EmbedText(words.word))
USING FAISS
""").df()

11-22-2023 20:41:42 WARNING[drop_object_executor:drop_object_executor.py:_handle_drop_index:0113] Index words_vector_idx does not exist, therefore cannot be dropped.
/home/oscar/.cache/pypoetry/virtualenvs/words-Cdd64V6x-py3.10/lib/python3.10/site-packages/sentence_transformers/SentenceTransformer.py:157: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  sentences_sorted = [sentences[idx] for idx in length_sorted_idx]


,0
0,Index words_vector_idx successfully added to t...


In [3]:
features = cursor.query(""" SELECT EmbedText(words.word) FROM words """).df()
features

/home/oscar/.cache/pypoetry/virtualenvs/words-Cdd64V6x-py3.10/lib/python3.10/site-packages/sentence_transformers/SentenceTransformer.py:157: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  sentences_sorted = [sentences[idx] for idx in length_sorted_idx]


,features
0,"[[-0.047473814, 0.07417954, -0.046912633, 0.04..."
1,"[[-0.048946433, 0.017024904, 0.0192541, 0.0421..."
2,"[[-0.06581226, 0.048192814, 0.03048122, 0.0248..."
3,"[[-0.021957867, 0.042924915, -0.041306958, 0.0..."
4,"[[-0.006681391, 0.0052218996, -0.0401461, 0.02..."
...,...
51581,"[[-0.058257163, 0.06376873, -0.113152884, -0.0..."
51582,"[[-0.058739588, -0.0023282839, -0.0433384, 0.0..."
51583,"[[-0.047309257, -0.056698933, -0.009764805, 0...."
51584,"[[-0.053118195, -0.008547983, 0.052530404, 0.0..."


In [16]:
features['features'][0][0][0]

-0.047473814

In [26]:
# convert dataframe to 50k x 768 numpy array
import numpy as np
f = np.array(features)
f.shape
f = np.empty((len(features), 384), dtype=np.float32)
for i, row in enumerate(features['features']):
    f[i] = row[0]
f

array([[-0.04747381,  0.07417954, -0.04691263, ...,  0.08421466,
         0.07241768,  0.09963789],
       [-0.04894643,  0.0170249 ,  0.0192541 , ...,  0.02739874,
         0.02628225,  0.05707146],
       [-0.06581226,  0.04819281,  0.03048122, ...,  0.00842909,
         0.11183214, -0.06755178],
       ...,
       [-0.04730926, -0.05669893, -0.00976481, ..., -0.02384533,
         0.03095684,  0.03388943],
       [-0.0531182 , -0.00854798,  0.0525304 , ...,  0.07888456,
         0.07785661, -0.00965249],
       [-0.11883838,  0.04829865, -0.00254808, ...,  0.12640947,
         0.04654906, -0.01571728]], dtype=float32)

In [27]:
import faiss
index = faiss.IndexFlatL2(384)
index.add(f)
index.ntotal

51586

In [12]:
test_query = "Joy"
cursor.query(f"""
SELECT * FROM words ORDER BY Similarity(EmbedText('{test_query}'), EmbedText(word)) LIMIT 15;
""").df()

,_row_id,word
0,4523,joy
1,16767,joys
2,18096,joyful
3,18967,joyous
4,44434,joyously
5,30361,joyfully
6,14178,happier
7,7077,delight
8,7147,smiling
9,3190,happy


In [28]:
# create a simple flask endpoint to serve the five most similar games
from flask import Flask, request, jsonify
app = Flask(__name__)
import sqlite3
# allow cross origin requests from any domain
from flask_cors import CORS
CORS(app)
import sentence_transformers
model = sentence_transformers.SentenceTransformer('all-MiniLM-L6-v2')
import time

@app.route('/get_similar_words', methods=['GET'])
def get_similar_professors():
    start = time.time()
    cursor2 = sqlite3.connect('./eva_db/evadb.db').cursor()
    query = request.args.get('query')
    query_vec = model.encode([query])  # Make sure this returns a 2D numpy array
    k = 15  # Define the number of results you want
    D, I = index.search(query_vec, k)
    I = I.squeeze()
    row_ids = [id + 1 for id in I]

    # Use SQLAlchemy to execute raw SQL
    # result = db.session.execute(sqlalchemy.text('SELECT name FROM profs WHERE rowid IN ({})'.format(','.join([str(i) for i in row_ids]))))
    result = cursor2.execute('SELECT _row_id, * FROM words WHERE rowid IN ({})'.format(','.join([str(i) for i in row_ids]))).fetchall()
    # map result to dict w/ fields name, research_profile, img_src, href
    result = [{'_row_id': r[0], 'word': r[1]} for r in result]
    # sort the result by the index at which res._row_id - 1 appears in I, the sorted indices by faiss
    result = sorted(result, key=lambda res: I.tolist().index(res['_row_id'] - 1))
    cursor2.connection.close()
    print('time elapsed: {}'.format(time.time() - start))
    return jsonify(result)


app.run(host="localhost", port=5002)

# this can be accessed as follows: http://localhost:5000/get_similar_games?query=fast-paced%20action%20game

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://localhost:5002
Press CTRL+C to quit
127.0.0.1 - - [30/Nov/2023 16:42:07] "GET /get_similar_words?query=undefined HTTP/1.1" 200 -


time elapsed: 0.015700340270996094


127.0.0.1 - - [30/Nov/2023 16:42:10] "GET /get_similar_words?query=error HTTP/1.1" 200 -


time elapsed: 0.03468179702758789


...

time elapsed: 0.05558514595031738


127.0.0.1 - - [30/Nov/2023 17:05:34] "GET /get_similar_words?query=unexplored HTTP/1.1" 200 -


time elapsed: 0.10011839866638184


127.0.0.1 - - [30/Nov/2023 17:51:39] "GET /get_similar_words?query=machine%20learning HTTP/1.1" 200 -


time elapsed: 0.05974292755126953
